# LangChain 실습

In [29]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(len(OPENAI_API_KEY))

164


In [ ]:
# !pip install openai langchain langchain-openai langchain-experimental langchainhub


   -------------------------- ------------- 2/3 [langchain-experimental]
   ---------------------------------------- 3/3 [langchain-experimental]



## 5-2. Agents

### Agents 사용 예
**hub.pull("hwchase17/react")**
- 랭체인 허브(LangChain Hub)에서 ReAct(Reasoning and Acting) 프롬프트 템플릿을 가져오는 명령어
- 이 프롬프트는 LLM이 단순히 답변만 하는 것이 아니라, **논리적으로 생각(Thought)**하고, 필요한 **도구(Tool)를 사용(Action)**하며, 그 **결과를 관찰(Observation)**하여 최종 답을 도출하도록 설계된 표준 템플릿
- 텍스트 기반의 추론(Thought/Action/Observation) 방식

In [2]:
import langchain
langchain.verbose = True # 내부적으로 어떤 일이 벌어지고 있는지 상세한 **로그(Log)**를 화면에 출력하도록 설정하는 전역 옵션

In [32]:
from langchain_classic import hub
from langchain_classic.agents import AgentExecutor, create_react_agent, load_tools
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)
tools = load_tools(["terminal"], allow_dangerous_tools=True)
prompt = hub.pull("hwchase17/react") # 랭체인 허브(LangChain Hub)에서 ReAct(Reasoning and Acting) 프롬프트 템플릿을 가져오는 명령어

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
result = agent_executor.invoke({"input": "sample_data 디렉터리에 있는 파일 목록을 알려줘, 터미널을 사용할 때 인코딩도 알아서 맞춰줘."})



> Entering new AgentExecutor chain...
Thought: sample_data 디렉터리에 있는 파일 목록을 확인하기 위해 터미널 명령어를 실행해야 한다. Windows 환경이므로 'dir' 명령어를 사용하고, 인코딩 문제를 방지하기 위해 PowerShell을 이용하거나 chcp 명령어로 코드 페이지를 설정할 수 있다. 우선 기본적으로 'dir sample_data' 명령어를 실행해 파일 목록을 확인하겠다.

Action: terminal
Action Input: chcp 65001Executing command:
 chcp 65001
Active code page: 65001


c:\ProgramData\anaconda3\Lib\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


Thought: 코드 페이지를 UTF-8(65001)로 설정했으니, 이제 sample_data 디렉터리의 파일 목록을 확인하는 명령어를 실행하겠다.

Action: terminal
Action Input: dir sample_dataExecuting command:
 dir sample_data
 Volume in drive C has no label.
 Volume Serial Number is 6C6F-4BFC

 Directory of c:\SeSAC_AI_3rd\study_python\AI\LLM

File Not Found
Thought: sample_data 디렉터리에 파일이 없거나 경로가 잘못된 것 같다. 현재 디렉터리 위치가 c:\SeSAC_AI_3rd\study_python\AI\LLM 인 것으로 보인다. sample_data 디렉터리가 이 경로 하위에 있는지 확인하거나, 상위 디렉터리에서 다시 확인해보겠다.

Action: terminal
Action Input: dir ..\sample_dataExecuting command:
 dir ..\sample_data
 Volume in drive C has no label.
 Volume Serial Number is 6C6F-4BFC

 Directory of c:\SeSAC_AI_3rd\study_python\AI

File Not Found
Thought: 상위 디렉터리에도 sample_data 폴더가 없고, 현재 경로 기준으로 sample_data 폴더가 존재하지 않는 것으로 보인다. 현재 위치에서 sample_data 폴더가 있는지 전체 경로를 확인하거나, 현재 위치에서 sample_data 폴더가 있는지 다시 한 번 확인해보겠다.

Action: terminal
Action Input: dir /s /b sample_dataExecuting command:
 dir /s /b sample_data
File Not Found
Thought: sample_data라는 이름의 디렉터리가 현

In [7]:
print(type(llm), type(prompt), type(agent), type(agent_executor))
print(type(tools), len(tools), type(tools[0]))
print(type(result), type(result["output"]))
print(result["input"])
print(result["output"])

<class 'langchain_openai.chat_models.base.ChatOpenAI'> <class 'langchain_core.prompts.prompt.PromptTemplate'> <class 'langchain_core.runnables.base.RunnableSequence'> <class 'langchain_classic.agents.agent.AgentExecutor'>
<class 'list'> 1 <class 'langchain_community.tools.shell.tool.ShellTool'>
<class 'dict'> <class 'str'>
sample_data 디렉터리에 있는 파일 목록을 알려줘, 터미널을 사용할 때 인코딩도 알아서 맞춰줘.
sample_data 디렉터리(C:\ProgramData\anaconda3\Lib\site-packages\matplotlib\mpl-data\sample_data) 내 파일 목록은 다음과 같습니다.

- axes_grid  
- data_x_x2_x3.csv  
- eeg.dat  
- embedding_in_wx3.xrc  
- goog.npz  
- grace_hopper.jpg  
- jacksboro_fault_dem.npz  
- logo2.png  
- membrane.dat  
- Minduka_Present_Blue_Pack.png  
- msft.csv  
- README.txt  
- s1045.ima.gz  
- Stocks.csv  
- topobathy.npz


### Tools

In [9]:
from langchain_classic.tools import Tool

def my_super_func(param):
    return "42"

tools = [
    Tool.from_function(
        func=my_super_func,
        name="The_Answer",
        description="생명, 우주, 그리고 모든 것에 대한 궁극적인 질문의 답"
    )
]

In [11]:
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
result = agent_executor.invoke({"input": "이 세계의 진리를 알려주세요"})
print(type(result), result)



> Entering new AgentExecutor chain...
Question: 이 세계의 진리를 알려주세요
Thought: 이 세계의 진리에 대해 묻는 질문이므로, '생명, 우주, 그리고 모든 것에 대한 궁극적인 질문의 답'을 제공하는 도구를 사용해 답을 찾아야겠다.
Action: The_Answer
Action Input: 이 세계의 진리42Thought: The answer "42" is a reference to the ultimate answer to life, the universe, and everything from Douglas Adams' "The Hitchhiker's Guide to the Galaxy." It symbolizes the ultimate truth or answer to the question about the world's truth.
Final Answer: 이 세계의 진리에 대한 궁극적인 답은 "42"입니다.

> Finished chain.
<class 'dict'> {'input': '이 세계의 진리를 알려주세요', 'output': '이 세계의 진리에 대한 궁극적인 답은 "42"입니다.'}


In [12]:
from langchain_openai import ChatOpenAI
from langchain_classic.prompts import PromptTemplate
from langchain_classic import LLMChain

summarize_template = """아래의 글을 결론만 한 마디로 요약해 주세요.
{input}
"""
summarize_prompt = PromptTemplate(input_variables=["input"], template=summarize_template)
chat = ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)
summarize_chain = LLMChain(llm=chat, prompt=summarize_prompt)
tools = [
    Tool.from_function(
        func=summarize_chain.run,
        name="Summarizer",
        description="Text summarizer"
    )
]

C:\Users\User\AppData\Local\Temp\ipykernel_13112\2395266583.py:10: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  summarize_chain = LLMChain(llm=chat, prompt=summarize_prompt)


In [13]:
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
text = """다음을 요약해 주세요.
안녕하세요! 저는 ChatGPT라고 불리는 AI 언어 모델입니다.
OpenAI가 개발한 GPT-3.5 아키텍처를 기반으로 합니다.
저는 자연어 이해와 생성을 전문으로 하며, 다양한 주제에 대한 질문에 답하거나, 대화를 나누는 것을 잘합니다.
제 트레이닝 데이터는 2021년 9월까지의 정보를 기반으로 하기 때문에, 그 이후의 사건에 대해서는 지식이 없습니다.
하지만, 가능한 한 도움을 드리기 위해 노력할 것입니다.
질문이나 대화, 정보 공유 등, 어떤 도움이든 편하게 말씀해 주세요! 잘 부탁드립니다."""
result = agent_executor.invoke({"input": text})



> Entering new AgentExecutor chain...
Thought: The input text is a self-introduction of ChatGPT, explaining its nature, capabilities, training data cutoff, and willingness to help. I will summarize this information concisely.
Action: Summarizer
Action Input: 안녕하세요! 저는 ChatGPT라고 불리는 AI 언어 모델입니다. OpenAI가 개발한 GPT-3.5 아키텍처를 기반으로 합니다. 저는 자연어 이해와 생성을 전문으로 하며, 다양한 주제에 대한 질문에 답하거나, 대화를 나누는 것을 잘합니다. 제 트레이닝 데이터는 2021년 9월까지의 정보를 기반으로 하기 때문에, 그 이후의 사건에 대해서는 지식이 없습니다. 하지만, 가능한 한 도움을 드리기 위해 노력할 것입니다. 질문이나 대화, 정보 공유 등, 어떤 도움이든 편하게 말씀해 주세요! 잘 부탁드립니다.저는 다양한 주제에 대해 도움을 드리는 AI 언어 모델입니다.Final Answer: 저는 OpenAI의 GPT-3.5 기반 AI 언어 모델 ChatGPT로, 자연어 이해와 생성에 능하며 2021년 9월까지의 정보를 바탕으로 다양한 질문과 대화에 도움을 드립니다.

> Finished chain.


In [14]:
print(type(result), result["output"])

<class 'dict'> 저는 OpenAI의 GPT-3.5 기반 AI 언어 모델 ChatGPT로, 자연어 이해와 생성에 능하며 2021년 9월까지의 정보를 바탕으로 다양한 질문과 대화에 도움을 드립니다.


### Function calling을 사용하는 OpenAI Functions Agent

**hub.pull("hwchase17/openai-functions-agent")**
- OpenAI 모델이 제공하는 함수 호출(Function Calling) 기능에 최적화된 프롬프트 템플릿을 가져오는 명령어
- hwchase17/react가 텍스트 기반의 추론(Thought/Action/Observation) 방식이라면, 이 프롬프트는 OpenAI의 API 자체 기능인 functions 파라미터를 활용하도록 설계

**hub.pull("hwchase17/openai-tools-agent")**
- OpenAI의 최신 모델들이 지원하는 Native Tool Calling (OpenAI Tools) 기능을 활용하기 위한 전용 프롬프트 템플릿
- 'openai-functions-agent'의 상위 호환 버전이며, 현재 랭체인에서 OpenAI 모델을 사용할 때 가장 권장되는 표준 방식

In [34]:
from langchain_classic.agents import AgentExecutor, create_openai_functions_agent
from langchain_classic.agents import load_tools
from langchain_classic import hub
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
tools = load_tools(["terminal"], allow_dangerous_tools=True)
prompt = hub.pull("hwchase17/openai-tools-agent")
# prompt = hub.pull("hwchase17/openai-functions-agent")

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "sample_data 디렉터리에 있는 파일 목록을 알려줘, 터미널을 사용할 때 인코딩도 알아서 맞춰서 해줘."})



> Entering new AgentExecutor chain...

Invoking: `terminal` with `{'commands': ['chcp 65001', 'dir sample_data']}`


Executing command:
 ['chcp 65001', 'dir sample_data']
Parameter format not correct - 65001;dir


c:\ProgramData\anaconda3\Lib\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(



Invoking: `terminal` with `{'commands': ['chcp 65001', 'dir sample_data']}`


Executing command:
 ['chcp 65001', 'dir sample_data']
Parameter format not correct - 65001;dir

Invoking: `terminal` with `{'commands': ['chcp 65001', 'dir sample_data']}`


Executing command:
 ['chcp 65001', 'dir sample_data']
Parameter format not correct - 65001;dir

Invoking: `terminal` with `{'commands': ['chcp 65001', 'dir', 'sample_data']}`


Executing command:
 ['chcp 65001', 'dir', 'sample_data']
Parameter format not correct - 65001;dir;sample_data

Invoking: `terminal` with `{'commands': ['chcp 65001', 'dir sample_data']}`


Executing command:
 ['chcp 65001', 'dir sample_data']
Parameter format not correct - 65001;dir

Invoking: `terminal` with `{'commands': ['chcp 65001', 'dir sample_data']}`


Executing command:
 ['chcp 65001', 'dir sample_data']
Parameter format not correct - 65001;dir

Invoking: `terminal` with `{'commands': ['chcp 65001', 'dir sample_data']}`


Executing command:
 ['chcp 65001'

{'input': 'sample_data 디렉터리에 있는 파일 목록을 알려줘, 터미널을 사용할 때 인코딩도 알아서 맞춰서 해줘.',
 'output': 'Agent stopped due to iteration limit or time limit.'}

### 여러 도구를 한 번에 사용
**load_tools(["ddg-search"])**
- 랭체인(LangChain)에서 DuckDuckGo 검색 엔진을 모델이 사용할 수 있는 '도구(Tool)'로 불러오는 명령어
- 이 명령어를 사용하면 LLM이 학습 데이터에 없는 최신 정보나 실시간 뉴스를 인터넷에서 검색하여 답변 가능
- Google 검색 API(Serper, SearchApi 등)는 대부분 유료이거나 API 키 발급이 필요하지만, DuckDuckGo는 별도의 키 없이 즉시 무료로 사용 가능
- 가볍고 빠르게 검색 결과를 반환

In [19]:
# !pip install duckduckgo_search

In [22]:
# !pip install -U ddgs


   -------------------------------- ------- 4/5 [ddgs]
   ---------------------------------------- 5/5 [ddgs]



In [20]:
import langchain
langchain.debug = True
langchain.verbose = False

In [23]:
from langchain_classic.agents import AgentExecutor, create_openai_functions_agent
from langchain_classic.agents import load_tools
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
tools = load_tools(["ddg-search"])
prompt = hub.pull("hwchase17/openai-functions-agent")

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
result = agent_executor.invoke({"input": "서울과 부산의 날씨를 알려줘"})



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': '서울 날씨'}`


We would like to show you a description here but the site won’t allow us. Hourly weather forecast in 서울특별시, 서울시, 대한민국. Check current conditions in 서울특별시, 서울시, 대한민국 with radar, hourly, and more. Weather Channel 및 Weather.com의 가장 정확한 최고온, 최저온, 강수 확률 정보가 제공되는 서울 특별시, 대한민국 10일 일기예보로 준비하세요 1 day ago · 사용자 친화적인 그래픽 인터페이스를 통해 서울특별시 의 풍속 및 풍향, 기압, 기온, 습도, 가시거리 등의 정보가 제공됩니다. 또한, 해당 일자의 해와 달의 일출 및 일몰 시간도 함께 안내됩니다. MSN 날씨과 (와) 함께 서울의 오늘, 오늘 밤, 내일에 대한 정확한 시간별 예보와 함께 10일간의 일일 예보 및 기상 레이더를 확인하세요. 강수량 ...
Invoking: `duckduckgo_search` with `{'query': '부산 날씨'}`


2025. 11. 10. · 내일 · 시각: 01시 · 날씨: 구름 많음 · 기온(체감온도) -4℃(-10℃) · 체감온도: -10℃ · 강수량: - · - · 강수확률: 20% · 바람: 북서풍 약간 강 4m/s ... 2025. 5. 9. · 부산지방기상청만의 날씨 서비스를 소개합니다ㅣ부산지방기상청 사무실&장비 공개! 11K views · 8 months ago. #기상청 #국민정책기자단 #부산지방기상청 ...more ... 6시간 전 · 낮 기온은 부산 15도, 김해 16도, 창원 17도, 사천 16도, 진주와 하동동 함안이 17도, 거창 14도 등으로 어제보다 5~9도 높겠습니다. 미세먼지 농도는 부산과 경

In [24]:
print(result["output"])

현재 서울과 부산의 날씨 정보는 다음과 같습니다.

서울: 현재 상세한 기상 정보는 제공되지 않았으나, 서울의 날씨는 대체로 맑거나 구름이 있을 수 있습니다.

부산: 부산은 대체로 맑고 추운 날씨가 이어지고 있으며, 낮 최고기온은 약 2도, 아침 최저기온은 영하 8도 정도로 예상됩니다. 바람은 북서풍 약간 강하게 불고 있습니다.

더 구체적인 시간별 기상 정보가 필요하시면 알려주세요!


### Function calling을 활용한 OurputParser, Extraction, Tagging
**create_extraction_chain(schema, llm)**
- 비정형 텍스트(뉴스 기사, 이메일, 채팅 메시지 등)에서 우리가 원하는 특정 정보만 구조화된 형태(JSON 등)로 뽑아내고 싶을 때 사용
- 랭체인이 업데이트되면서 현재는 create_extraction_chain보다 더 발전된 .with_structured_output() 함수 사용을 권장

In [25]:
import json
from langchain_openai import ChatOpenAI
from langchain_classic.chains import create_extraction_chain

schema = {
    "properties": {
        "person_name": {"type": "string"},
        "person_height": {"type": "integer"},
        "person_hair_color": {"type": "string"},
        "dog_name": {"type": "string"},
        "dog_breed": {"type": "string"},
    },
    "required": ["person_name", "person_height"],
}
text = """
Alex is 5 feet tall. Claudia is 1 feet taller Alex and jumps higher than him. Claudia is a brunette and Alex is blonde.
Alex's dog Frosty is a labrador and likes to play hide and seek.
"""
chat = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
chain = create_extraction_chain(schema, chat)
people = chain.invoke(text)

C:\Users\User\AppData\Local\Temp\ipykernel_13112\632485.py:20: LangChainDeprecationWarning: LangChain has introduced a method called `with_structured_output` thatis available on ChatModels capable of tool calling.You can read more about the method here: <https://docs.langchain.com/oss/python/langchain/models#structured-outputs>.
  chain = create_extraction_chain(schema, chat)


In [33]:
print(type(people))
print(json.dumps(people, indent=2))

<class 'dict'>
{
  "input": "\nAlex is 5 feet tall. Claudia is 1 feet taller Alex and jumps higher than him. Claudia is a brunette and Alex is blonde.\nAlex's dog Frosty is a labrador and likes to play hide and seek.\n",
  "text": [
    {
      "person_name": "Alex",
      "person_height": 5,
      "person_hair_color": "blonde"
    },
    {
      "person_name": "Claudia",
      "person_height": 6,
      "person_hair_color": "brunette"
    },
    {
      "person_name": "Frosty",
      "dog_breed": "labrador"
    }
  ]
}


### Evaluation

In [27]:
from langchain_classic.evaluation import load_evaluator
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
evaluator = load_evaluator("qa", eval_llm=chat)
result = evaluator.evaluate_strings(
    input="""나는 시장에 가서 사과 10개를 샀어.
    사과 2개를 이웃에게 주고, 2개를 수리공에게 주었어.
    그리고 사과 5개를 더 사서 1개는 내가 먹었어.
    나는 몇 개의 사과를 가지고 있었니?""",
    prediction="""먼저 사과 10개로 시작했어.
    이웃에게 2개, 수리공에게 2개를 나누어 주었으므로 사과가 6개가 남았어.
    그런 다음 사과 5개를 더 사서 이제 사과가 11개가 되었어.
    마지막으로 사과 1개를 먹었으므로 사과 10개가 남게 돼.""",
    reference="10개",
)

In [28]:
print(result)

{'reasoning': 'CORRECT', 'value': 'CORRECT', 'score': 1}
